In [15]:
## GPT를 활용해서 Few shot inference를 수행합니다.
## 하나의 data 생성에 30초정도 걸립니다... korquad의 unique한 context 9000개를 활용해서 
## 문제를 제작할 경우 약 16시간정도의 시간이 걸립니다.
import openai
from datasets import load_dataset
import pandas as pd
import numpy as np
from ast import literal_eval


data = pd.read_csv('train.csv')
korquad = load_dataset('KorQuAD/squad_kor_v1')
context = list(set(korquad['train']['context']))
client = openai.OpenAI(
    api_key="YOUR API KEY"
)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
def generate(context):
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {
                "role": "system",
                "content": "주어진 context를 다음과 같은 형식으로 문제로 만들어주세요.",
            },
            {
                "role": "user",
                "content": (
                    f"paragraph : {data['paragraph'][0]} problems : {data['problems'][0]} "
                    f"paragraph : {data['paragraph'][1]} problems : {data['problems'][2]} "
                    f"paragraph : {data['paragraph'][1000]} problems : {data['problems'][1000]} "
                    f"paragraph : {data['paragraph'][1030]} problems : {data['problems'][1030]} "
                    f"paragraph : {context} problems : "
                ),
            },
        ],
    )
    response = response.choices[0].message.content
    row = {
        "id": f"yongari_generate_{int(str(abs(hash(context)))[:5])}",
        "paragraph": context,
        "problems": response.replace("\n", " "),  # \n 제거
        "question_plus": np.nan,
    }
    return row


In [17]:
contexts = list(set(korquad['train']['context']))

In [ ]:
from tqdm import tqdm
generated = []
for context in tqdm(contexts):
    generated.append(generate(context))

In [22]:
generated = pd.DataFrame(generated)

In [23]:
import re
from ast import literal_eval

# '''를 제거하는 정규식
def cleaning(text):
    cleaned_text = re.sub(r"```|'''", "", text)
    cleaned_text = re.sub(r"json", "", cleaned_text)
    
    return cleaned_text
# data['problems']에서 cleaning 적용
generated['problems'] = generated['problems'].apply(cleaning)


In [36]:
for i , item in generated.iterrows():
    try:
        literal_eval(item['problems'])
    except:
        print(i,'에서 오류발생',)

In [10]:
# 생성을 하더라도 오류가 있는 행이 존재합니다. 정규식을 추가해서 지우던가 행을 지우던가 csv파일을 직접 보고 지우던가 하시면 됩니다.
generated.iloc[2446]

"paragraph: 이 논쟁은 정밀 조사가 끝날 때까지 멈추지 않았다. 히틀러는 9월에 태풍 작전을 시작하여 중앙집단군이 소련의 방어선 깊이 침투하여 스몰렌스크 동쪽에서 독일군의 병력을 낭비하면 안 된다고 주장했다. 또한, 중앙집단군은 끊임없이 취약한 우측 측면으로의 공세를 시작했고 최고의 계산으로 가을 장마가 시작하는 10월 중순 모스크바 앞에 도착한다고 계산했다. 결국, 스탑스카는 도시의 최종 방어에 참여할 10개 군을 확보하고 모스크바에 주둔시켰으며 1941년 12월 방어를 시작하고 1942년 1월 반격을 시작한다. 이 군과 상관없이 히틀러는 태풍 작전을 시작했을 것이다. 이들이 효과적으로 모스크바 공세를 늦추고 짧게 공세를 하는 동안 작전은 실제로 발전되었으며 이렇게 했으면 독일은 이달 초 모스크바 공격이 가능했을 것이다. 또한, 만약 후자의 경우라면 이들은 소련군 60만명의 포로도 함께 병력에 포함시킬 수 있었을 것이다.  problems: {'question': '1941년 중반 독일군의 전략에 대한 설명으로 옳지 않은 것은 무엇인가?', 'choices': ['태풍 작전을 통해 중앙집단군이 소련의 방어선을 깊숙이 침투했다.', '히틀러는 중앙집단군이 스몰렌스크 동쪽에서 병력을 낭비하는 것을 우려했다.', '스트랙스 카는 모스크바의 최종 방어를 준비했다.', '모스크바 주변에 도착한 독일군은 소련군과의 대규모 전투 없이 간단히 도시를 점령할 수 있었다.', '히틀러는 태풍 작전을 시작함으로써 모스크바 공세를 빠르게 마치려 했다.'], 'answer': 4}"

In [14]:
# 저는 몇몇개는 정규식으로, 정규식 짜기 힘든건 직접 지웠습니다. csv파일 들어가서 cmd+f를 치고 행 id를 입력해서 지우면 쉽게 지울 수 있습니다.
generated.iloc[2446]

id                                          yongari_generate_62210
paragraph        이 논쟁은 정밀 조사가 끝날 때까지 멈추지 않았다. 히틀러는 9월에 태풍 작전을 시...
problems         {'question': '1941년 중반 독일군의 전략에 대한 설명으로 옳지 않은 ...
question_plus                                                  NaN
Name: 2446, dtype: object

In [6]:
generated['test'] = generated['problems'].apply(lambda x: literal_eval(x))

In [7]:
# 이 과정에서도 오류가 발생할 수 있습니다. 
for idx, row in generated.iterrows():
    try:
        problems = literal_eval(row['problems'])
        record = {
        'id': row['id'],
        'paragraph': row['paragraph'],
        'question': problems['question'],
        'choices': problems['choices'],
        'answer': problems.get('answer', None),
        "question_plus": problems.get('question_plus', None),
        }
    except:
        print(idx,'에서 오류 발생')

In [17]:
# 결과물 중 일부가 복수 정답을 갖는 경우가 있습니다. 정답이 하나인 경우만 뽑도록 필터링합니다.
generated['len'] = generated['test'].apply(lambda x: len(str(x['answer'])))

In [22]:
generated = generated[generated['len'] == 1]

In [26]:
processed = generated[['id', 'paragraph', 'problems', 'question_plus']]
generated.to_csv('train+generated.csv', index = False)
generated[generated['id'].str.startswith('yongari')].to_csv('generated_from_korquad.csv', index = False)


In [ ]:
d